In [4]:
import pickle
import numpy as np
import torch
import pytorch_lightning as pl
from pytorch_lightning import Trainer
import loss
%load_ext autoreload
%autoreload 2

In [5]:
with open("data.sample.25k.pickle", "rb") as f:
    data = pickle.load(f)

In [6]:
class Dataset(torch.utils.data.Dataset):
    
    """Simple torch dataset class"""
    def __init__(self, data):

        self.data = data

    def __len__(self):

        return len(self.data)

    def __getitem__(self, index):

        with torch.no_grad():
             
            vec1_np, vec2_np, str1, str2, _ = self.data[index]
            
            vec1, vec2, str1, str2, pair_id = self.data[index]            
            vec1, vec2 = torch.from_numpy(vec1_np).float(), torch.from_numpy(vec2_np).float()
            return (vec1, vec2, str1, str2, pair_id)

In [25]:
class Siamese(pl.LightningModule):

    def __init__(self, X_train, X_dev, dim = 50, batch_size = 1000):
        super(Siamese, self).__init__()
        self.l = torch.nn.Linear(768,50)
        
        self.train_data = Dataset(X_train)
        self.dev_data = Dataset(X_dev)
        self.train_gen = torch.utils.data.DataLoader(self.train_data, batch_size = batch_size, drop_last = False, shuffle=True)
        self.dev_gen = torch.utils.data.DataLoader(self.dev_data, batch_size = batch_size, drop_last = False, shuffle=True)
        self.loss_fn = loss.BatchHardTripletLoss(final = "plus")
        
        self.acc = None
        self.optimizer = torch.optim.Adam(self.parameters(), weight_decay = 1e-6)
        
    def forward(self, x1, x2):

          h1 = self.l(x1)
          h2 = self.l(x2)
         
          return h1, h2
 
    def train_network(self, num_epochs):
    
      trainer = Trainer(max_nb_epochs = num_epochs, min_nb_epochs = num_epochs, show_progress_bar = True)
      trainer.fit(self)

      return self.acc   
      
    def get_weights(self):
    
        return self.l.weight.data.numpy()
    
    def training_step(self, batch, batch_nb):
        # REQUIRED
        x1, x2, str1, str2, ids = batch
        h1, h2 = self.forward(x1, x2)
        loss_val =  self.loss_fn(h1, h2, str1, str2, ids, index=0, evaluation = False)
        
        return {'loss': loss_val[0]}
        

    def validation_step(self, batch, batch_nb):
    
        # OPTIONAL
        x1, x2, str1, str2, ids = batch
        h1, h2 = self.forward(x1, x2)
        loss_val =  self.loss_fn(h1, h2, str1, str2, ids, index=batch_nb, evaluation = True)
        return {'val_loss': loss_val[0]}

    def validation_end(self, outputs):
        # OPTIONAL    
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        print("Loss is {}".format(avg_loss))
        return {'avg_val_loss': avg_loss}

    def configure_optimizers(self):
        # REQUIRED
        return torch.optim.Adam(self.parameters(), weight_decay = 1e-4)

    @pl.data_loader
    def train_dataloader(self):
        return self.train_gen

    @pl.data_loader
    def val_dataloader(self):
        # OPTIONAL
        # can also return a list of val dataloaders
        return self.dev_gen

In [ ]:
s = Siamese(data[:20000], data[20000:], batch_size = 512)
s.train_network(num_epochs = 25)

INFO:root:      Name                  Type Params
0        l                Linear   38 K
1  loss_fn  BatchHardTripletLoss    0  
Epoch 1:   8%|▊         | 4/50 [00:00<00:02, 21.85batch/s, batch_nb=3, loss=0.187, v_nb=18]

Loss is 0.24401716887950897


Epoch 2:   8%|▊         | 4/50 [00:00<00:01, 26.37batch/s, batch_nb=3, loss=0.133, v_nb=18]  

Loss is 0.12430765479803085


Epoch 3:   8%|▊         | 4/50 [00:00<00:01, 24.95batch/s, batch_nb=3, loss=0.129, v_nb=18]  

...

Epoch 23:   8%|▊         | 4/50 [00:00<00:01, 26.17batch/s, batch_nb=3, loss=nan, v_nb=18]  

Loss is 0.1919921487569809


Epoch 24:   8%|▊         | 4/50 [00:00<00:01, 26.57batch/s, batch_nb=3, loss=nan, v_nb=18]  

Loss is 0.1925114095211029


Epoch 25:   0%|          | 0/50 [00:00<00:01, 28.44batch/s, batch_nb=39, loss=nan, v_nb=18] 

In [8]:
exit()